In [2]:
%autosave 300
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [3]:
import os

os.chdir("../")
print(os.getcwd())

c:\workspace\EagSession1\eag_agentic_rag\url_rag


In [4]:
from IPython.display import display, HTML, Markdown
from dotenv import load_dotenv

In [4]:
# Load environment variables
load_dotenv()

# Clear SSL_CERT_FILE environment variable if set
if "SSL_CERT_FILE" in os.environ:
    del os.environ["SSL_CERT_FILE"]

In [5]:
from utility.llm_provider import default_llm
from utility.embedding_provider import OpenAIEmbeddingProvider
from utility.utils import read_yaml_file

urllib3 SSL patched successfully
httpx Client patched to disable SSL verification
httpx AsyncHTTPTransport patched to disable SSL verification
✅ SSL verification completely disabled at all levels
SSL_CERT_FILE: None
PYTHONHTTPSVERIFY: 0
OPENAI_VERIFY_SSL_CERTS: false
OPENAI_API_SKIP_VERIFY_SSL: true
✅ SSL test success: 401


In [6]:
llm = default_llm.chat_model
embedder = OpenAIEmbeddingProvider().embeddings

In [7]:
print(llm.invoke("what is the capital of France?"))
print(len(embedder.embed_query("what is the capital of France?")))

content='The capital of France is Paris.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 14, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f5bdcc3276', 'id': 'chatcmpl-BUHkVM02Ghurwph9B4PevIbc26eMI', 'finish_reason': 'stop', 'logprobs': None} id='run-ae0382ca-0281-4fe6-b295-748fc98de293-0' usage_metadata={'input_tokens': 14, 'output_tokens': 8, 'total_tokens': 22, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
1536


In [ ]:
# read config
config = read_yaml_file("utility/config.yaml")
print(config)

In [ ]:
history_index_name = config["history_index_name"]
history_index_name = os.path.join(os.getcwd(), history_index_name)
print(history_index_name)

In [10]:
import uuid
from client.memory import FaissConversationStore

In [ ]:
# conv_id = uuid.uuid4()
# store = FaissConversationStore(
#     embedder, index_folder=history_index_name, reset_index=True
# )

In [12]:
# messages = [
#     {"sender": "human", "content": "Hello!"},
#     {"sender": "ai", "content": "Hi, how can I help you?"},
#     {"sender": "human", "content": "Tell me a joke."},
#     {"sender": "ai", "content": "Why did the chicken cross the road?"},
# ]
# store.store_conversation(str(conv_id), messages)
# print(store.get_conversation(str(conv_id)))

In [13]:
# # new conversation with same id
# messages = [
#     {"sender": "human", "content": "Tell me a joke about a space alien"},
#     {"sender": "ai", "content": "Why did the space alien cross the road?"},
#     {"sender": "human", "content": "What is the capital of France?"},
#     {"sender": "ai", "content": "The capital of France is Paris."},
# ]
# store.store_conversation(str(conv_id), messages)
# print(store.get_conversation(str(conv_id)))

#### Next steps 
- Perception code which will be used prepare the question based on the user query and chat history
- Decision making code which will be used to decide the action based on the question and chat history
- Action code which will be used to execute the action
- Memory code which will be used to store the chat history
- Client code which will be used to coordinate the conversation between the perception, decision making, action and memory


##### Perception code

In [14]:
from pydantic import BaseModel, Field
from typing import Literal, Optional
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableSequence
import json

In [15]:
class WebContentSearch(BaseModel):
    enhanced_user_query: str = Field(
        description="The enhanced user query to be searched on the web for similar content stored in the memory"
    )
    no_of_results: int = Field(
        1,
        description="The number of results to be returned from the web search corresponding to the user query",
    )

In [16]:
system_prompt = """
You are an intelligent assistant designed to refine user queries using current input and prior context. Your goal is to construct a precise, context-aware enhanced query that reflects the user’s true intent.

Your responsibilities are:
1. Analyze the current user message to understand the core request.
2. Examine the chat history (a list of messages with sender and content) to identify:
   - Any contextually relevant information.
   - Follow-ups or references to previous discussions.
3. Determine whether the current query is:
   a. A continuation of a previous topic.
   b. A new, unrelated query.

Based on this analysis, follow the appropriate path:

**If the current message relates to previous topics:**
- Incorporate relevant prior context into the enhanced query.
- Clarify ambiguous references or pronouns using information from the chat history.
- Resolve under-specified requests by grounding them in previous conversations.

**If the message is a new topic:**
- Focus solely on the current message.
- Do not infer or inject unrelated context from chat history.

**Process for forming the enhanced user query:**
1. Carefully review the current message.
2. Analyze the chat history (most recent last) to extract relevant past content.
3. Identify if the user is building on a previous conversation.
4. Integrate any necessary clarifications, references, or details from the chat history.
5. Output a concise, context-enriched, unambiguous enhanced query.

**Inputs:**
- `chat_history`: A list of dictionaries, each with keys `sender` and `content`.
    {chat_history}
- `user_query`: The latest message from the user.
    {user_query}

**Output Format:**
{format_instructions}
"""

In [17]:
def get_perception_chain(llm) -> RunnableSequence:
    """
    Creates and returns the perception chain for extracting travel search parameters.

    Args:
        default_llm: The default LLM to use for the chain.

    Returns:
        A LangChain runnable sequence that takes a user query and chat history,
        and returns a TravelSearch object.
    """

    # Set up a parser
    parser = PydanticOutputParser(pydantic_object=WebContentSearch)

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "The user query is: {user_query}"),
        ]
    )

    prompt = prompt.partial(format_instructions=parser.get_format_instructions())

    # Create and return the chain
    return prompt | llm | parser

In [ ]:
conv_id = uuid.uuid4()
store = FaissConversationStore(
    embedder, index_folder=history_index_name, reset_index=True
)

In [ ]:
chat_history = store.get_conversation_as_lc_messages(str(conv_id))
print(chat_history)

In [29]:
user_query = "What is role of helm in kubernetes?"

In [30]:
perception_chain = get_perception_chain(llm)

In [ ]:
result = perception_chain.invoke(
    {"user_query": user_query, "chat_history": chat_history}
)
print(result)

In [33]:
# create the history object
messages = [
    {"sender": "human", "content": user_query},
    {"sender": "ai", "content": result.enhanced_user_query},
]
store.store_conversation(str(conv_id), messages)

In [ ]:
user_query = "Give me two articles on for the same topic?"
chat_history = store.get_conversation_as_lc_messages(str(conv_id))
print(chat_history)

result = perception_chain.invoke(
    {"user_query": user_query, "chat_history": chat_history}
)
print(result)

In [36]:
# create the history object
messages = [
    {"sender": "human", "content": user_query},
    {"sender": "ai", "content": result.enhanced_user_query},
]
store.store_conversation(str(conv_id), messages)

In [ ]:
chat_history = store.get_conversation_as_lc_messages(str(conv_id))
print(chat_history)

In [38]:
# create a function where we will pass the history object and a list of conversation and the chain
# and it will keep processing the conversation and return the enhanced user query


def process_conversation_and_enhance_query(
    history_store, conversation_list, chain, conv_id
):
    """
    Processes a list of user queries (conversation_list) using the provided chain and history store.
    For each user query, it retrieves the chat history, invokes the chain, stores the conversation,
    and returns a list of enhanced user queries.

    Args:
        history_store: The object responsible for storing and retrieving conversation history.
        conversation_list: List of user queries (strings) to process.
        chain: The chain object with an 'invoke' method.
        conv_id: The conversation id to store the conversation.

    Returns:
        List of enhanced user queries (one for each user query in conversation_list).
    """
    enhanced_queries = []
    for user_query in conversation_list:
        # Retrieve chat history for the conversation
        chat_history = history_store.get_conversation_as_lc_messages(str(conv_id))
        # Invoke the chain to get the result
        result = chain.invoke({"user_query": user_query, "chat_history": chat_history})
        # Store the conversation
        messages = [
            {"sender": "human", "content": user_query},
            {"sender": "ai", "content": result.enhanced_user_query},
        ]
        history_store.store_conversation(str(conv_id), messages)
        # Collect the enhanced user query
        enhanced_queries.append(result.enhanced_user_query)
    return enhanced_queries

In [ ]:
conv_id = uuid.uuid4()
memory_store = FaissConversationStore(
    embedder, index_folder=history_index_name, reset_index=True
)
conversation_list = [
    "What is the capital of France?",
    "Tell me more about its history.",
    "List two famous landmarks in that city.",
]
enhanced_queries = process_conversation_and_enhance_query(
    memory_store, conversation_list, perception_chain, conv_id
)
print(enhanced_queries)

##### Decision and action code

- For the constraints of mcp we cannot use the decision node in jupter notebook , so we have executed the decision node in the test_client.py file and have saved the output in the test_client_output.pkl file
- We will try to debug the action node here


In [5]:
decison_op = "client/test_client_output.json"
import json
with open(decison_op, "r") as f:
    data = json.load(f)

In [6]:
data

{'response': {'content': '',
  'tool_calls': [{'name': 'web_vector_search',
    'id': 'call_FWHEcSZih0LGVsBIvpGLRC5w',
    'type': 'tool_call',
    'args': {'request': {'query': 'What is the purpose and benefits of using Helm in software development or Kubernetes management?',
      'k': 1}}}],
  'model': 'gpt-4o-2024-08-06',
  'finish_reason': 'tool_calls',
  'usage': {'total_tokens': 247, 'input_tokens': 210, 'output_tokens': 37}},
 'messages': [{'type': 'HumanMessage',
   'content': 'What is the object of using helm?',
   'tool_calls': []},
  {'type': 'AIMessage',
   'content': '',
   'tool_calls': [{'name': 'web_vector_search',
     'id': 'call_FWHEcSZih0LGVsBIvpGLRC5w',
     'type': 'tool_call',
     'args': {'request': {'query': 'What is the purpose and benefits of using Helm in software development or Kubernetes management?',
       'k': 1}}}]}],
 'tools': [{'name': 'web_vector_search',
   'description': '\n    Perform advanced web search using vector database and return the res

In [7]:
# read the pickle file
import pickle
with open("client/test_client_output.pickle", "rb") as f:
    data = pickle.load(f)


In [8]:
response = data["response"]
msg_objs = data["msg_objs"]
tools = data["tools"]

In [9]:
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_FWHEcSZih0LGVsBIvpGLRC5w', 'function': {'arguments': '{"request":{"query":"What is the purpose and benefits of using Helm in software development or Kubernetes management?","k":1}}', 'name': 'web_vector_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 210, 'total_tokens': 247, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f5bdcc3276', 'id': 'chatcmpl-BUI6CtcIxvtTuQ6RcYcaaa61VViIc', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-dfc381cd-69fe-42ce-8787-50be4eb0d4f6-0', tool_calls=[{'name': 'web_vector_search', 'args': {'request': {'query': 'What is the purpose and benefits of using Helm in software development or K

In [10]:
from notebooks.tool_example import web_vector_search

In [11]:
tools = [web_vector_search]

In [16]:
tool_dict = {tool.name: tool for tool in tools}

In [17]:
tool_dict

{'web_vector_search': StructuredTool(name='web_vector_search', description='Search the knowledge base for information on any topic.\n\n    This tool must be used for ANY request requiring factual information.\n\n    Args:\n        request: The WebSearchRequest containing query and k parameters', args_schema=<class 'langchain_core.utils.pydantic.web_vector_search'>, func=<function web_vector_search at 0x00000219FB2A47C0>)}

In [20]:
selected_tool = tool_dict["web_vector_search"]
await selected_tool.ainvoke(
    {"request": {"query": "What is the capital of France?", "k": 1}}
)

WebSearchResponse(results=['Found 1 results for: What is the capital of France? (schema validated). The knowledge base indicates What is the capital of France? is a package manager for Kubernetes that simplifies application deployment and management.'])

In [68]:
# tool_dict = {tool.name: tool for tool in tools}
# print(tool_dict)

In [69]:
# if hasattr(response, "tool_calls") and response.tool_calls:
#     # Handle tool calls
#     print(f"Processing {len(response.tool_calls)} tool call(s)...")
#     print(response.tool_calls)

In [70]:
# for tool_call in response.tool_calls:
#     print(tool_call)
#     if tool_call["name"] in tool_dict:
#         selected_tool = tool_dict[tool_call["name"]]
#         print(f"Executing tool: {tool_call['name']}")

#         # Process the arguments - make a copy to avoid modifying the original
#         tool_args = dict(tool_call["args"])

#         # Ensure the arguments match the expected schema
#         print(f"Original tool args: {tool_args}")

#         # Get the schema from the tool if available
#         tool_schema = None
#         if hasattr(selected_tool, "args_schema"):
#             tool_schema = selected_tool.args_schema
#             schema_name = (
#                 tool_schema.__name__
#                 if hasattr(tool_schema, "__name__")
#                 else type(tool_schema).__name__
#             )
#             print(
#                 f"Tool schema found: {schema_name} and the type is {type(tool_schema)}"
#             )

In [71]:
# tool_args

In [72]:
# from langchain_core.messages import ToolMessage

# messages = []
# tool_output = selected_tool.invoke(tool_args)

# messages.append(ToolMessage(content=tool_output, tool_call_id=tool_call["id"]))

In [73]:
# messages

In [77]:
from typing import List, Dict, Any
from langchain_core.messages import ToolMessage

async def process_tool_calls(
    tool_calls: List[Dict], tools: List[Any]
) -> List[ToolMessage]:
    """
    Process tool calls by invoking the appropriate tool with arguments.
    Handles schema validation and nested argument structures.
    If invocation with nested dict fails, fallback to simple dict.

    Args:
        tool_calls: List of tool calls from the LLM response.
        tools: List of available tools.

    Returns:
        List of ToolMessage objects with the results.
    """
    messages = []
    tool_dict = {tool.name: tool for tool in tools}

    for tool_call in tool_calls:
        tool_name = tool_call.get("name")
        tool_id = tool_call.get("id")
        tool_args = tool_call.get("args", {})
        print(f"\nProcessing tool call: {tool_name} (id: {tool_id})")
        if tool_name not in tool_dict:
            print(f"Tool {tool_name} not found in available tools")
            messages.append(
                ToolMessage(content=f"Tool {tool_name} not found.", tool_call_id=tool_id)
            )
            continue

        selected_tool = tool_dict[tool_name]
        print(f"Executing tool: {tool_name}")

        # Print original tool args
        print(f"Original tool args: {tool_args}")

        # Handle schema if present
        tool_schema = getattr(selected_tool, "args_schema", None)
        if tool_schema is not None:
            schema_name = getattr(tool_schema, "__name__", type(tool_schema).__name__)
            print(f"Tool schema found: {schema_name} ({type(tool_schema)})")

        # Try nested dict first, fallback to simple dict if it fails
        tried_fallback = False
        try:
            print(f"Calling tool with args: {tool_args}")
            if hasattr(selected_tool, "ainvoke"):
                tool_output = await selected_tool.ainvoke(tool_args)
            else:
                tool_output = selected_tool.invoke(tool_args)
            messages.append(
                ToolMessage(content=tool_output, tool_call_id=tool_id)
            )
        except Exception as e:
            print(f"Tool invocation failed with nested dict: {e}")
            # Fallback: If tool_args is a dict with a single key (e.g. 'request'), try flattening
            fallback_args = None
            if isinstance(tool_args, dict) and len(tool_args) == 1:
                only_key = next(iter(tool_args))
                if isinstance(tool_args[only_key], dict):
                    fallback_args = tool_args[only_key]
            # If not, try passing tool_args as is (in case it's already flat)
            if fallback_args is None and isinstance(tool_args, dict):
                fallback_args = tool_args
            if fallback_args is not None and not tried_fallback:
                try:
                    print(f"Retrying tool invocation with fallback args: {fallback_args}")
                    if hasattr(selected_tool, "ainvoke"):
                        tool_output = await selected_tool.ainvoke(fallback_args)
                    else:
                        tool_output = selected_tool.invoke(fallback_args)
                    messages.append(
                        ToolMessage(content=tool_output, tool_call_id=tool_id)
                    )
                    continue
                except Exception as e2:
                    error_msg = f"Error executing tool {tool_name} (fallback): {e2}"
                    print(error_msg)
                    messages.append(
                        ToolMessage(content=error_msg, tool_call_id=tool_id)
                    )
            else:
                error_msg = f"Error executing tool {tool_name}: {e}"
                print(error_msg)
                messages.append(
                    ToolMessage(content=error_msg, tool_call_id=tool_id)
                )

    return messages


In [78]:
messages = await process_tool_calls(response.tool_calls, tools)


Processing tool call: web_vector_search (id: call_FWHEcSZih0LGVsBIvpGLRC5w)
Executing tool: web_vector_search
Original tool args: {'request': {'query': 'What is the purpose and benefits of using Helm in software development or Kubernetes management?', 'k': 1}}
Tool schema found: web_vector_search (<class 'pydantic._internal._model_construction.ModelMetaclass'>)
Calling tool with args: {'request': {'query': 'What is the purpose and benefits of using Helm in software development or Kubernetes management?', 'k': 1}}


In [79]:
messages

[ToolMessage(content="results=['Found 1 results for: What is the purpose and benefits of using Helm in software development or Kubernetes management? (schema validated). The knowledge base indicates What is the purpose and benefits of using Helm in software development or Kubernetes management? is a package manager for Kubernetes that simplifies application deployment and management.']", tool_call_id='call_FWHEcSZih0LGVsBIvpGLRC5w')]

######################################### END OF FILE #########################################